# Part 1 - Data Preprocessing

### Importing the Libraries

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

### Importing the Datasets

In [2]:
movies = pd.read_csv('../input/users-movies-ratings/ml-1m/ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
users = pd.read_csv('../input/users-movies-ratings/ml-1m/ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings = pd.read_csv('../input/users-movies-ratings/ml-1m/ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')

In [3]:
print("Movies dataset shape : ", movies.shape)
print("Ratings dataset shape : ", ratings.shape)
print("Users dataset shape : ", users.shape)

Movies dataset shape :  (3883, 3)
Ratings dataset shape :  (1000209, 4)
Users dataset shape :  (6040, 5)


In [4]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [6]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### Preparing the training and test set (use ml-100k dataset)

In [7]:
training_set = pd.read_csv('../input/users-movies-ratings/ml-100k/ml-100k/u1.base', delimiter='\t')
test_set = pd.read_csv('../input/users-movies-ratings/ml-100k/ml-100k/u1.test', delimiter='\t') 

In [8]:
print("Training Shape ; ", training_set.shape)
print("Test Shape ; ", test_set.shape)

Training Shape ;  (79999, 4)
Test Shape ;  (19999, 4)


### Converting dataset into numpy array

In [9]:
training_set = np.array(training_set, dtype='int')
test_set = np.array(test_set, dtype='int')

### Getting number of users and movies

In [10]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [11]:
print("No. of Users : ", nb_users)
print("No. of Movies : ", nb_movies)

No. of Users :  943
No. of Movies :  1682


### Converting the data in array with users in lines and movies in columns

In [12]:
def convert(data):
    new_data = []
    
    for id_users in range(1, nb_users+1):     #we add 1 because range doesn't incluse upper bound
        id_movies = data[:, 1][data[:,0] == id_users]     #we get all the movies id for each user
        id_ratings = data[:, 2][data[:,0] == id_users]     #we get ratings of seen movies of each user
        ratings = np.zeros(nb_movies)     #creating numpy array of no. of movies size array of zeros
        ratings[id_movies - 1] = id_ratings     #sub 1 because index start with 0 and id_movies start with value 1
        new_data.append(list(ratings))     #append list of ratings to new_list
        
    return new_data     #returning new_data list

In [13]:
training_data = convert(training_set)
test_data = convert(test_set)

### So now we have a 943 rows(users) and 1682 columns(movies)

In [14]:
len(training_data), len(training_data[0])

(943, 1682)

### Converting the data into Torch tensors

In [15]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [16]:
type(training_set), type(test_set)

(torch.Tensor, torch.Tensor)

### Converting the ratings into 1 (Liked) or 0 (Not Liked)

In [17]:
training_set[training_set == 0] = -1     #assign -1 to the movies that user didn't seen
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

### Creating the Architecture of RBM(Restricted Boltzmann Machine)

In [18]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):
        wx = torch.mm(x, self.W.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):
        wy = torch.mm(y, self.W)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [19]:
nv = len(training_set[0])
nh = 100
batch_size = 100

# Part 2 - Training the RBM

In [20]:
rbm = RBM(nv, nh)

In [21]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user : id_user + batch_size]
        v0 = training_set[id_user : id_user + batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
        s += 1.
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: tensor(0.0850)
epoch: 2 loss: tensor(0.0839)
epoch: 3 loss: tensor(0.1222)
epoch: 4 loss: tensor(0.1161)
epoch: 5 loss: tensor(0.1128)
epoch: 6 loss: tensor(0.1117)
epoch: 7 loss: tensor(0.1228)
epoch: 8 loss: tensor(0.1161)
epoch: 9 loss: tensor(0.1417)
epoch: 10 loss: tensor(0.0950)


# Part 3 - Testing the RBM 

In [22]:
test_loss = 0
s = 0
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.0854)
